# Train Hassner's CNN

reference: https://talhassner.github.io/home/projects/cnn_agegender/CVPR2015_CNN_AgeGenderEstimation.pdf

In [1]:
# import necessary libraries
# import os
import tensorflow as tf


from hassner_cnn import get_hassner_cnn

In [2]:
# random seed
seed = 0
tf.random.set_seed(seed)

In [3]:
# configs and params
# os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
try:
    gpu = tf.config.list_physical_devices('GPU')[0]
    tf.config.experimental.set_memory_growth(gpu, True)
except:
    print('GPU not detected by Tensorflow')

data_dir = '/home/burntice/3_data/Adience/sorted_dataset/'
image_size = (256, 256)
batch_size = 32

initial_learning_rate = 10**-3
learning_rate_after_10k = 10**-4

initial_num_epochs = 10000
num_epochs_per_step = 50  # for usage with early stopping

In [4]:
# load train data
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    label_mode='int',
    image_size=image_size,
    batch_size=batch_size,
    seed=seed,
    validation_split=0.2,
    subset='training',
)

Found 17492 files belonging to 17 classes.
Using 13994 files for training.


In [5]:
# load validation data
val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    label_mode='int',
    image_size=image_size,
    batch_size=batch_size,
    seed=seed,
    validation_split=0.2,
    subset='validation',
)

Found 17492 files belonging to 17 classes.
Using 3498 files for validation.


In [6]:
class_names = val_dataset.class_names
num_classes = len(class_names)

print(class_names)

['female(0,3)', 'female(15,24)', 'female(25,37)', 'female(38,47)', 'female(4,7)', 'female(48,59)', 'female(60)', 'female(8,14)', 'female(no_age)', 'male(0,3)', 'male(15,24)', 'male(25,37)', 'male(38,47)', 'male(4,7)', 'male(48,59)', 'male(60)', 'male(8,14)']


In [7]:
model = get_hassner_cnn((256, 256, 3), num_classes)

model.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=initial_learning_rate),
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy'],
)

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 250, 250, 96)      14208     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 124, 124, 96)      0         
_________________________________________________________________
lambda (Lambda)              (None, 124, 124, 96)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 120, 120, 256)     614656    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 59, 59, 256)       0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 59, 59, 256)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 57, 57, 384)       8

In [8]:
# train for (initial_num_epochs) epochs first
train_history = model.fit(
    train_dataset,
    batch_size=batch_size,
    epochs=initial_num_epochs,
    verbose=0,  # not showing training details
    validation_data=val_dataset,
)

results = model.evaluate(
    val_dataset,
    batch_size=batch_size,
    return_dict=True,
)

KeyboardInterrupt: 